In [2]:
import numpy as np
import pandas as pd
import os
import csv
import re
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
class OpenAIClient:
    def __init__(self, api_key, model="gpt-4o"):
        self.client = OpenAI(api_key=api_key)
        self.model = model
    
    def generate(self, prompt):
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1  # 일관성을 위해 낮은 temperature
            )
            return response.choices[0].message.content
        except Exception as e:
            raise Exception(f"OpenAI API error: {str(e)}")

# 사용 방법
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
gpt_client = OpenAI(api_key=openai_api_key)
model_client = OpenAIClient(api_key=openai_api_key, model="gpt-4")


In [4]:
df = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/In-Network_Providers_Update.csv')
ground_truth = df["Providers"].dropna().str.strip().tolist()
df.head()

,Providers,Source,Links
0,Aarkansas Medicaid (FFS),X,NaN
1,Aetna,html,https://www.aetna.com/cpb/medical/data/100_199...
2,Aetna | Texas Health,X,NaN
3,Aetna Better Health,NaN,NaN
4,Aetna Better Health Illinois,X,NaN


In [5]:
baseline_gpt1 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_baseline_experiment1/provider.csv')
baseline_gpt2 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_baseline_experiment2/provider.csv')
baseline_gpt3 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_baseline_experiment3/provider.csv')
explicit_gpt1 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_explicit_source_experiment1/provider.csv')
explicit_gpt2 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_explicit_source_experiment2/provider.csv')
explicit_gpt3 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/ChatGPT_explicit_source_experiment3/provider.csv')

baseline_perplexity1 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_baseline_experiment1/provider.csv')
baseline_perplexity2 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_baseline_experiment2/provider.csv')
baseline_perplexity3 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_baseline_experiment3/provider.csv')
explicit_perplexity1 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_explicit_source_experiment1/provider.csv')
explicit_perplexity2 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_explicit_source_experiment2/provider.csv')
explicit_perplexity3 = pd.read_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_explicit_source_experiment3/provider.csv')

In [6]:
print(np.mean([len(baseline_gpt1), len(baseline_gpt2), len(baseline_gpt3)]), np.mean([len(explicit_gpt1), len(explicit_gpt2), len(explicit_gpt3)]),np.mean([len(baseline_perplexity1), len(baseline_perplexity2), len(baseline_perplexity3)]), np.mean([len(explicit_perplexity1), len(explicit_perplexity2), len(explicit_perplexity3)]) )

184.0 94.33333333333333 7.666666666666667 8.0


In [7]:
def normalize_provider_name(name):
    name = name.strip()
    
    # 1. 특수문자 정규화 (LLM 룰 반영)
    name = re.sub(r'[–—]', '-', name)  # em-dash, en-dash → hyphen
    name = re.sub(r'\s+', ' ', name)   # 다중 공백 → 단일 공백
    
    # 2. Oscar 정규화 (LLM 룰 반영)
    if name.lower().strip() == "oscar":
        name = "Oscar Health Insurance"
    
    # 3. Fix garbled characters
    name = re.sub(r"\?\?\s*Medicaid", "- Medicaid", name, flags=re.IGNORECASE)

    # 4. Kansas City → Kansas
    name = name.replace("Kansas City", "Kansas")

    # 5. Wellpoint (Amerigroup ...) → Wellpoint
    if re.search(r"Wellpoint\s*\(Amerigroup.*?\)", name, flags=re.IGNORECASE):
        name = "Wellpoint"

    # 6. Wellpoint (XX) → Wellpoint XX
    match = re.match(r"Wellpoint\s*\((\w{2})\)", name)
    if match:
        name = f"Wellpoint {match.group(1)}"

    # 7. State abbreviation mapping
    state_abbrev = {
        "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
        "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
        "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
        "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA",
        "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT",
        "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
        "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
        "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
        "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
        "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
    }

    # 8. These keep the full state name - check first
    preserve_state_full = [
        "Amerigroup", "Anthem BCBS", "BCBS", "Blue Cross", "CareSource", "Healthy Blue", "Molina Healthcare"
    ]
    
    # Check if this should preserve full state name
    should_preserve = False
    for prefix in preserve_state_full:
        if re.search(rf"^{prefix}\s+of\s+[A-Za-z ]+", name, flags=re.IGNORECASE):
            should_preserve = True
            break
    
    if not should_preserve:
        # 9. AmeriHealth Caritas State → AmeriHealth Caritas XX
        for state, abbr in state_abbrev.items():
            if re.fullmatch(rf"AmeriHealth Caritas {state}", name, flags=re.IGNORECASE):
                name = f"AmeriHealth Caritas {abbr}"
                break
            if re.fullmatch(rf"Wellpoint {state}", name, flags=re.IGNORECASE):
                name = f"Wellpoint {abbr}"
                break

        for state, abbr in state_abbrev.items():
            # BCBS California → BCBS CA
            if re.fullmatch(rf"BCBS {state}", name, flags=re.IGNORECASE):
                name = f"BCBS {abbr}"
                break
            # BCBS (CA) → BCBS CA  
            if re.fullmatch(rf"BCBS \({abbr}\)", name, flags=re.IGNORECASE):
                name = f"BCBS {abbr}"
                break
        
        # 11. X of State → X (for non-preserved cases)
        name = re.sub(r"\bof\s+[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b", "", name)

    # 12. Clean-up
    name = name.replace("&", "and")
    name = re.sub(r"\s+", " ", name)  # 다시 한번 공백 정리
    name = re.sub(r"\s*\(.*?\)", "", name)  # 괄호 내용 제거

    return name.strip()

In [8]:
def compute_provider_metrics_multi_level(predicted, ground_truth, llm_evaluator=None):
    """
    Compute precision and recall at three levels:
    1. Raw (no normalization)
    2. Normalized
    3. LLM-based evaluation
    """
    
    def _compute_metrics(pred_list, gt_list, level_name):
        """Helper function to compute metrics for a given level"""
        if not pred_list or not gt_list:
            return {
                f"{level_name}_ground_truth_count": len(gt_list),
                f"{level_name}_llm_returned_count": len(pred_list),
                f"{level_name}_common_count": 0,
                f"{level_name}_missing_count": len(gt_list),
                f"{level_name}_extra_count": len(pred_list),
                f"{level_name}_Common_Providers": [],
                f"{level_name}_Missing_Providers": gt_list,
                f"{level_name}_Extra_Providers": pred_list,
                f"{level_name}_Precision (%)": 0.0,
                f"{level_name}_Recall (%)": 0.0,
            }
        
        pred_set = set(pred_list)
        gt_set = set(gt_list)
        common = pred_set & gt_set
        missing = gt_set - pred_set
        extra = pred_set - gt_set
        
        precision = len(common) / len(pred_set) * 100 if pred_set else 0
        recall = len(common) / len(gt_set) * 100 if gt_set else 0
        
        return {
            f"{level_name}_ground_truth_count": len(gt_set),
            f"{level_name}_llm_returned_count": len(pred_set),
            f"{level_name}_common_count": len(common),
            f"{level_name}_missing_count": len(missing),
            f"{level_name}_extra_count": len(extra),
            f"{level_name}_Common_Providers": sorted(list(common)),
            f"{level_name}_Missing_Providers": sorted(list(missing)),
            f"{level_name}_Extra_Providers": sorted(list(extra)),
            f"{level_name}_Precision (%)": round(precision, 2),
            f"{level_name}_Recall (%)": round(recall, 2),
        }
    
    def _compute_llm_metrics(predicted, ground_truth, llm_evaluator):
        """
        Use LLM to evaluate provider lists
        """
        if not predicted or not ground_truth:
            
            return {
                "LLM_ground_truth_count": len(ground_truth),
                "LLM_llm_returned_count": len(predicted),
                "LLM_common_count": 0,
                "LLM_missing_count": len(ground_truth),
                "LLM_extra_count": len(predicted),
                "LLM_Common_Providers": [],
                "LLM_Missing_Providers": list(ground_truth),
                "LLM_Extra_Providers": list(predicted),
                "LLM_Precision (%)": 0.0,
                "LLM_Recall (%)": 0.0,
            }

        llm_results = llm_evaluator.evaluate_provider_lists(predicted, ground_truth)
        if "LLM_Common_Providers" not in llm_results:
            llm_results["LLM_Common_Providers"] = []
            
        return llm_results
        
    
    results = {}
    
    # 1. Raw analysis (no normalization)
    raw_results = _compute_metrics(predicted, ground_truth, "Raw")
    results.update(raw_results)
    
    # 2. Normalized analysis
    normalized_predicted = [normalize_provider_name(x) for x in predicted]
    normalized_ground_truth = [normalize_provider_name(x) for x in ground_truth]
    normalized_results = _compute_metrics(normalized_predicted, normalized_ground_truth, "Normalized")
    results.update(normalized_results)
    
    # 3. LLM-based evaluation
    if llm_evaluator:
        llm_results = _compute_llm_metrics(predicted, ground_truth, llm_evaluator)
        results.update(llm_results)
    
    return results

In [9]:
class LLMProviderEvaluator:
    def __init__(self, model_client, log_csv_path=None):
        self.model_client = model_client
        self.log_csv_path = log_csv_path or f"llm_evaluation_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        self._initialize_csv_log()

    def _initialize_csv_log(self):
        if not os.path.exists(self.log_csv_path):
            with open(self.log_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([
                    'timestamp', 'ground_truth_count', 'predicted_count', 
                    'llm_raw_response', 'parsed_common_count', 'parsed_common_providers', 'parsed_missing_providers', 
                    'parsed_extra_providers', 'final_precision', 'final_recall', 'parsing_error'
                ])

    def _log_to_csv(self, ground_truth, predicted, llm_response, parsed_result, final_metrics, parsing_error=None):
        try:
            with open(self.log_csv_path, 'a', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    len(ground_truth or []), len(predicted or []),
                    (llm_response or '').replace('\n', '\\n').replace('\r', '\\r'), 
                    parsed_result.get('common_count', 0),
                    '|'.join(parsed_result.get('common', [])),
                    '|'.join(parsed_result.get('missing', [])),
                    '|'.join(parsed_result.get('extra', [])),
                    final_metrics.get('LLM_Precision (%)', 0),
                    final_metrics.get('LLM_Recall (%)', 0),
                    str(parsing_error) if parsing_error else ''
                ])
        except Exception as e:
            print(f"Warning: Failed to log to CSV: {e}")



    def evaluate_provider_lists(self, predicted, ground_truth):
        """
        LLM evaluates two provider lists and returns standard metrics
        """
        if not predicted or not ground_truth:
            
            predicted = predicted or []
            ground_truth = ground_truth or []
            empty_result = {
                "LLM_ground_truth_count": len(ground_truth),
                "LLM_llm_returned_count": len(predicted),
                "LLM_common_count": 0,
                "LLM_missing_count": len(ground_truth),
                "LLM_extra_count": len(predicted),
                "LLM_Common_Providers": [],
                "LLM_Missing_Providers": list(ground_truth),
                "LLM_Extra_Providers": list(predicted),
                "LLM_Precision (%)": 0.0,
                "LLM_Recall (%)": 0.0,
            }

            self._log_to_csv(ground_truth, predicted, "EMPTY_LISTS", {}, empty_result)
            return empty_result
        
        prompt = f"""
You are an expert evaluator tasked with comparing two healthcare provider lists to calculate precision and recall metrics.

Your job is to:
1. Compare the Predicted list (experimental results) against the Ground Truth list
2. Identify which providers represent the same healthcare insurance entity
3. Calculate standard evaluation metrics

Matching Rules:
- Medicaid suffix is a KEY DIFFERENTIATOR: "Dean Health Plan" ≠ "Dean Health Plan – Medicaid" (different entities)
- Special characters/spacing/capitalization differences are IGNORED: "AmeriHealth" = "Amerihealth", "Anthem – Medicaid" = "Anthem - Medicaid"
- Company name extensions are considered same entity: "Oscar" = "Oscar Health Insurance"
- United Healthcare plans are ALL DIFFERENT entities: "United Healthcare" ≠ "United Healthcare Community Plan" ≠ "United Healthcare Community Plan (DC)"
- BCBS regional plans are different entities: "BCBS Texas" ≠ "BCBS Minnesota"

Examples:
- MATCH: "BCBS California" ↔ "BCBS CA" ↔ "BCBS (CA)" (same region, different abbreviations)
- MATCH: "Oscar" ↔ "Oscar Health Insurance"
- MATCH: "Anthem – Medicaid" ↔ "Anthem - Medicaid" (dash vs em-dash)
- NO MATCH: "Dean Health Plan" ↔ "Dean Health Plan – Medicaid" (different entities)
- NO MATCH: "BCBS Texas" ↔ "BCBS Minnesota" (different regions)

Step-by-step process:
1. First, identify all exact matches between the two lists following the rules above
2. Count matched providers (common_count)
3. List unmatched Ground Truth providers (missing)
4. List unmatched Predicted providers (extra)

Ground Truth ({len(ground_truth)} providers):
{chr(10).join([f"{i+1}. {gt}" for i, gt in enumerate(ground_truth)])}

Predicted ({len(predicted)} providers):
{chr(10).join([f"{i+1}. {pred}" for i, pred in enumerate(predicted)])}

Please provide your analysis and results in the following format:
Common Count: [number]
Common Providers: [list]
Missing Providers: [list]
Extra Providers: [list]
        """
        
        try:
            response = self.model_client.generate(prompt)
            parsed_result = self._parse_llm_response(response)
            
            # Convert to standard metric format
            precision = parsed_result['common_count'] / len(predicted) * 100 if predicted else 0
            recall = parsed_result['common_count'] / len(ground_truth) * 100 if ground_truth else 0

            final_metrics = {
                "LLM_ground_truth_count": len(ground_truth),
                "LLM_llm_returned_count": len(predicted),
                "LLM_common_count": parsed_result['common_count'],
                "LLM_missing_count": len(parsed_result['missing']),
                "LLM_extra_count": len(parsed_result['extra']),
                "LLM_Common_Providers": sorted(parsed_result['common']),
                "LLM_Missing_Providers": sorted(parsed_result['missing']),
                "LLM_Extra_Providers": sorted(parsed_result['extra']),
                "LLM_Precision (%)": round(precision, 2),
                "LLM_Recall (%)": round(recall, 2),
            }
            self._log_to_csv(ground_truth, predicted, response, parsed_result, final_metrics)
            return final_metrics
        
        except Exception as e:
            # Fallback in case of LLM error
            error_result = {
                "LLM_ground_truth_count": len(ground_truth),
                "LLM_llm_returned_count": len(predicted),
                "LLM_common_count": 0,
                "LLM_missing_count": len(ground_truth),
                "LLM_extra_count": len(predicted),
                "LLM_Common_Providers": [],
                "LLM_Missing_Providers": list(ground_truth),
                "LLM_Extra_Providers": list(predicted),
                "LLM_Precision (%)": 0.0,
                "LLM_Recall (%)": 0.0,
                "LLM_Error": str(e),
            }
        
            self._log_to_csv(ground_truth, predicted, f"ERROR: {str(e)}", {}, error_result, parsing_error=str(e))
        return error_result
    
    def _parse_llm_response(self, response):
        """
        Parse LLM response to extract structured data
        """     
        try:
            # Extract common count
            common_match = re.search(r'Common Count:\s*(\d+)', response, re.IGNORECASE)
            common_count = int(common_match.group(1)) if common_match else 0

            # Extract common providers - 추가
            common_providers_match = re.search(r'Common Providers:\s*\[(.*?)\]', response, re.DOTALL | re.IGNORECASE)
            if common_providers_match:
                common_text = common_providers_match.group(1).strip()
                common_providers = [item.strip().strip('"\'') for item in common_text.split(',') if item.strip() and item.strip() != 'None']
            else:
                common_providers = []
            
            # Extract missing providers
            missing_match = re.search(r'Missing Providers:\s*\[(.*?)\]', response, re.DOTALL | re.IGNORECASE)
            if missing_match:
                missing_text = missing_match.group(1).strip()
                missing = [item.strip().strip('"\'') for item in missing_text.split(',') if item.strip() and item.strip() != 'None']
            else:
                missing = []
            
            # Extract extra providers
            extra_match = re.search(r'Extra Providers:\s*\[(.*?)\]', response, re.DOTALL | re.IGNORECASE)
            if extra_match:
                extra_text = extra_match.group(1).strip()
                extra = [item.strip().strip('"\'') for item in extra_text.split(',') if item.strip() and item.strip() != 'None']
            else:
                extra = []
            
            return {
                'common_count': common_count,
                'common': common_providers,
                'missing': missing,
                'extra': extra
            }
            
        except Exception as e:
            # If parsing fails, return empty structure
            return {
                'common_count': 0,
                'common': [],
                'missing': [],
                'extra': []
            }


In [10]:
experiments = {
    "baseline_gpt1": baseline_gpt1,
    "baseline_gpt2": baseline_gpt2,
    "baseline_gpt3": baseline_gpt3,
    "explicit_gpt1": explicit_gpt1,
    "explicit_gpt2": explicit_gpt2,
    "explicit_gpt3": explicit_gpt3,
    "baseline_perplexity1": baseline_perplexity1,
    "baseline_perplexity2": baseline_perplexity2,
    "baseline_perplexity3": baseline_perplexity3,
    "explicit_perplexity1": explicit_perplexity1,
    "explicit_perplexity2": explicit_perplexity2,
    "explicit_perplexity3": explicit_perplexity3,
}

experiments_gpt = {k: v for k, v in experiments.items() if 'gpt' in k.lower()}
experiments_perplexity = {k: v for k, v in experiments.items() if 'perplexity' in k.lower()}

In [11]:
# LLM evaluator 생성
llm_evaluator = LLMProviderEvaluator(model_client)


In [12]:
def calculate_union_metrics(row, ground_truth):
    """Calculate union precision/recall from three methodologies"""
    
    # Extract common providers from each method (handle string/list conversion)
    raw_common = set(row.get('Raw_Common_Providers', []) or [])
    normalized_common = set(row.get('Normalized_Common_Providers', []) or [])
    llm_common = set(row.get('LLM_Common_Providers', []) or [])
    
    # Extract extra providers from each method
    raw_extra = set(row.get('Raw_Extra_Providers', []) or [])
    normalized_extra = set(row.get('Normalized_Extra_Providers', []) or [])
    llm_extra = set(row.get('LLM_Extra_Providers', []) or [])
    
    # Union of all common providers
    union_common = raw_common | normalized_common | llm_common
    
    # Calculate union metrics
    ground_truth_set = set(ground_truth)
    predicted_total = (raw_common | raw_extra) | (normalized_common | normalized_extra) | (llm_common | llm_extra)
    
    union_precision = len(union_common) / len(predicted_total) * 100 if predicted_total else 0
    union_recall = len(union_common) / len(ground_truth_set) * 100 if ground_truth_set else 0
    
    return {
        'Union_Common_Count': len(union_common),
        'Union_Common_Providers': sorted(list(union_common)),
        'Union_Precision (%)': round(union_precision, 2),
        'Union_Recall (%)': round(union_recall, 2)
    }


In [13]:
# GPT 실험 결과 분석 (Multi-level)
gpt_rows = []
for name, df_pred in experiments_gpt.items():
    pred_list = (
        df_pred["Providers"]
        .dropna()
        .astype(str)
        .str.strip()
        .tolist()  # normalize 제거 - multi-level에서 처리
    )
    metrics = compute_provider_metrics_multi_level(pred_list, ground_truth, llm_evaluator)
    metrics["experiment"] = name
    gpt_rows.append(metrics)

metrics_gpt_df = pd.DataFrame(gpt_rows)

# Perplexity 실험 결과 분석 (Multi-level)
perp_rows = []
for name, df_pred in experiments_perplexity.items():
    pred_list = (
        df_pred["Providers"]
        .dropna()
        .astype(str)
        .str.strip()
        .tolist()  # normalize 제거 - multi-level에서 처리
    )
    metrics = compute_provider_metrics_multi_level(pred_list, ground_truth, llm_evaluator)
    metrics["experiment"] = name
    perp_rows.append(metrics)

metrics_perplexity_df = pd.DataFrame(perp_rows)

# Union 메트릭스 계산 (GPT)
union_metrics_gpt = []
for _, row in metrics_gpt_df.iterrows():
    union_metrics = calculate_union_metrics(row, ground_truth)
    union_metrics_gpt.append(union_metrics)

union_gpt_df = pd.DataFrame(union_metrics_gpt)
metrics_gpt_df = pd.concat([metrics_gpt_df, union_gpt_df], axis=1)

# Union 메트릭스 계산 (Perplexity)
union_metrics_perp = []
for _, row in metrics_perplexity_df.iterrows():
    union_metrics = calculate_union_metrics(row, ground_truth)
    union_metrics_perp.append(union_metrics)

union_perp_df = pd.DataFrame(union_metrics_perp)
metrics_perplexity_df = pd.concat([metrics_perplexity_df, union_perp_df], axis=1)

# CSV 파일로 저장
metrics_gpt_df.to_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/gpt_experiments_metrics.csv', index=False)
metrics_perplexity_df.to_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/perplexity_experiments_metrics.csv', index=False)
print(metrics_gpt_df.head())

print("✅ GPT Experiments Metrics (Multi-level):")
print(metrics_gpt_df[['experiment', 'Raw_Precision (%)', 'Raw_Recall (%)', 
                     'Normalized_Precision (%)', 'Normalized_Recall (%)',
                     'LLM_Precision (%)', 'LLM_Recall (%)',
                     'Union_Precision (%)', 'Union_Recall (%)']])

print("\n✅ Perplexity Experiments Metrics (Multi-level):")
print(metrics_perplexity_df[['experiment', 'Raw_Precision (%)', 'Raw_Recall (%)', 
                            'Normalized_Precision (%)', 'Normalized_Recall (%)',
                            'LLM_Precision (%)', 'LLM_Recall (%)',
                            'Union_Precision (%)', 'Union_Recall (%)']])

   Raw_ground_truth_count  Raw_llm_returned_count  Raw_common_count  \
0                     239                     165                45   
1                     239                     191               148   
2                     239                     196               153   
3                     239                     160               125   
4                     239                      85                78   

   Raw_missing_count  Raw_extra_count  \
0                194              120   
1                 91               43   
2                 86               43   
3                114               35   
4                161                7   

                                Raw_Common_Providers  \
0  [Aetna, Aetna Better Health, Aetna Better Heal...   
1  [Aetna, Aetna Better Health, Aetna | Texas Hea...   
2  [Aetna, Aetna Better Health, Aetna | Texas Hea...   
3  [Aetna, Allina Health | Aetna, AmeriHealth Car...   
4  [Aetna, American Indian Health Program (AIH

In [14]:
metrics_all_df = pd.concat([metrics_gpt_df, metrics_perplexity_df], ignore_index=True)
metrics_all_df.to_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/all_experiments_metrics.csv', index=False)

# 그룹 이름 뽑기 (experiment 이름에서 숫자 제거)
metrics_all_df["group"] = metrics_all_df["experiment"].str.replace(r"\d+$", "", regex=True)

# 그룹별 평균 계산 (4가지 레벨 - Union 추가)
grouped_means = (
    metrics_all_df.groupby("group")[["Raw_Precision (%)", "Raw_Recall (%)",
                                    "Normalized_Precision (%)", "Normalized_Recall (%)",
                                    "LLM_Precision (%)", "LLM_Recall (%)",
                                    "Union_Precision (%)", "Union_Recall (%)"]]  # Union 추가
    .mean()
    .round(2)
    .reset_index()
)

# 출력 및 저장
print("\n📊 모델별 평균 Precision / Recall (Multi-level + Union)")
print(grouped_means)

# 그룹별 평균 결과도 저장
grouped_means.to_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/grouped_means_metrics.csv', index=False)



📊 모델별 평균 Precision / Recall (Multi-level + Union)
                 group  Raw_Precision (%)  Raw_Recall (%)  \
0         baseline_gpt              60.94           48.26   
1  baseline_perplexity              46.67            1.40   
2         explicit_gpt              86.45           33.06   
3  explicit_perplexity             100.00            3.35   

   Normalized_Precision (%)  Normalized_Recall (%)  LLM_Precision (%)  \
0                     72.91                  63.69              66.67   
1                     46.67                   1.57              46.67   
2                     95.84                  41.78              94.51   
3                    100.00                   3.76             100.00   

   LLM_Recall (%)  Union_Precision (%)  Union_Recall (%)  
0           53.98                74.47             87.17  
1            1.40                43.86              1.40  
2           37.52                97.36             57.60  
3            3.35                96.67   

In [15]:
# 모델별 평균 계산
metrics_all_df["model"] = metrics_all_df["experiment"].apply(
    lambda x: "gpt" if "gpt" in x.lower() else "perplexity"
)

model_means = (
    metrics_all_df.groupby("model")[["Raw_Precision (%)", "Raw_Recall (%)",
                                    "Normalized_Precision (%)", "Normalized_Recall (%)",
                                    "LLM_Precision (%)", "LLM_Recall (%)",
                                    "Union_Precision (%)", "Union_Recall (%)"]]  # Union 추가
    .mean()
    .round(2)
    .reset_index()
)

# 출력 및 저장
print("\n📊 GPT vs Perplexity 평균 성능 비교 (Multi-level)")
print(model_means)

model_means.to_csv('/home/cptaswadu/RESCUE-n8n/insurance/results/Providers Retrieval/model_means_metrics.csv', index=False)


📊 GPT vs Perplexity 평균 성능 비교 (Multi-level)
        model  Raw_Precision (%)  Raw_Recall (%)  Normalized_Precision (%)  \
0         gpt              73.70           40.66                     84.37   
1  perplexity              73.33            2.37                     73.33   

   Normalized_Recall (%)  LLM_Precision (%)  LLM_Recall (%)  \
0                  52.74              80.59           45.75   
1                   2.66              73.33            2.37   

   Union_Precision (%)  Union_Recall (%)  
0                85.92             72.39  
1                70.26              2.58  


In [16]:
# 레벨별 성능 비교를 위한 추가 분석
print("\n📈 레벨별 성능 개선 분석:")
for model in ["gpt", "perplexity"]:
    model_data = model_means[model_means["model"] == model].iloc[0]
    print(f"\n{model.upper()} 모델:")
    print(f"  Raw → Normalized: Precision {model_data['Raw_Precision (%)']}% → {model_data['Normalized_Precision (%)']}%")
    print(f"  Raw → Normalized: Recall {model_data['Raw_Recall (%)']}% → {model_data['Normalized_Recall (%)']}%")
    print(f"  Normalized → LLM: Precision {model_data['Normalized_Precision (%)']}% → {model_data['LLM_Precision (%)']}%")
    print(f"  Normalized → LLM: Recall {model_data['Normalized_Recall (%)']}% → {model_data['LLM_Recall (%)']}%")


📈 레벨별 성능 개선 분석:

GPT 모델:
  Raw → Normalized: Precision 73.7% → 84.37%
  Raw → Normalized: Recall 40.66% → 52.74%
  Normalized → LLM: Precision 84.37% → 80.59%
  Normalized → LLM: Recall 52.74% → 45.75%

PERPLEXITY 모델:
  Raw → Normalized: Precision 73.33% → 73.33%
  Raw → Normalized: Recall 2.37% → 2.66%
  Normalized → LLM: Precision 73.33% → 73.33%
  Normalized → LLM: Recall 2.66% → 2.37%
